**Phase 2**

The process began with getting detailed information and images of artifacts that is called global egyptian musem website. Using BeautifulSoup, we then extracted relevant images saved into a folder, and descriptions of artifacts from websites. Later stored in a structured format in json.

In [ ]:
#DONT PUT THIS
#this code is for testing and printing the html structure of web
import urllib.request
from bs4 import BeautifulSoup

# Base URL
base_url = 'https://www.globalegyptianmuseum.org/detail.aspx?id=1'
# header
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

# Parse HTML
try:
    request = urllib.request.Request(base_url, headers=headers)
    response = urllib.request.urlopen(request)
    soup = BeautifulSoup(response, "html.parser")

    # Print the HTML content]
    # print(soup.prettify())

except urllib.error.HTTPError as e:
    print(e)

In [ ]:
!pip install requests beautifulsoup4


In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import json

In [ ]:
#download image from url
def download_image(url, save_path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)
    else:
        print(f'Failed to download image from {url}')

#scrape data for artifact
def scrape_artifact(artifact_id, image_folder):
    # Construct the URL
    url = f'https://www.globalegyptianmuseum.org/detail.aspx?id={artifact_id}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract artifact title
    title = soup.find('h1').text.strip()

    # Find the image URL
    img_tag = soup.find('table', {'id': 'imgtable'}).find('img')
    if img_tag and 'src' in img_tag.attrs:
        image_relative_url = img_tag['src']
        image_url = f'https://www.globalegyptianmuseum.org/{image_relative_url}'
    else:
        image_url = None
        print(f'No image found for artifact ID {artifact_id}')

    # Prepare image path
    image_name = f'artifact_{artifact_id}.jpg'
    image_path = os.path.join(image_folder, image_name)

    # Download image if available
    if image_url:
        download_image(image_url, image_path)

    # Extract description
    description = soup.find('p', {'id': 'description'}).text.strip()

    # Extract details
    details = {}
    rows = soup.select('table#datatable tr')
    for row in rows:
        columns = row.find_all('td')
        if len(columns) == 2:
            key = columns[0].find('h4').text.strip()
            value = columns[1].text.strip()
            details[key] = value

    # Create dictionary
    artifact = {
        'title': title,
        'image': image_path if image_url else None,
        'description': description,
        'details': details
    }
    return artifact

# Create folder
image_folder = 'images'
os.makedirs(image_folder, exist_ok=True)

# Scrape artifacts from the website
artifacts = []
start_id = 1
end_id = 50 #CHANGE LATER

for artifact_id in range(start_id, end_id + 1):
    try:
        artifact = scrape_artifact(artifact_id, image_folder)
        artifacts.append(artifact)
    except Exception as e:
        print(f'Error scraping artifact ID {artifact_id}: {e}')

# Save to JSON file
with open('artifacts.json', 'w', encoding='utf-8') as f:
    json.dump(artifacts, f, ensure_ascii=False, indent=4)


Successfully downloaded image: images/artifact_1.jpg
Successfully downloaded image: images/artifact_2.jpg
Successfully downloaded image: images/artifact_3.jpg
Successfully downloaded image: images/artifact_4.jpg
Successfully downloaded image: images/artifact_5.jpg
Successfully downloaded image: images/artifact_6.jpg
Successfully downloaded image: images/artifact_7.jpg
Successfully downloaded image: images/artifact_8.jpg
Successfully downloaded image: images/artifact_9.jpg
Successfully downloaded image: images/artifact_10.jpg
Successfully downloaded image: images/artifact_11.jpg
Successfully downloaded image: images/artifact_12.jpg
Successfully downloaded image: images/artifact_13.jpg
Successfully downloaded image: images/artifact_14.jpg
Successfully downloaded image: images/artifact_15.jpg
Successfully downloaded image: images/artifact_16.jpg
Successfully downloaded image: images/artifact_17.jpg
Successfully downloaded image: images/artifact_18.jpg
Successfully downloaded image: images

**Phase 3**

Used nlp to generate enhanced information for each artifact by using spacy with a summary of key named entities identified.

In [ ]:
import spacy
# Ensure the SpaCy model is installed
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("Downloading SpaCy 'en_core_web_sm' model...")
    from spacy.cli import download

    download("en_core_web_sm")
    nlp = spacy.load("en_core_web_sm")

In [ ]:
#load json file
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

#Enhance description
def generate_description(description, nlp):
    doc = nlp(description)

    # Collect named entities
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    # Generate single sentence characteristics(CHANGE)
    entity_summary = ", ".join([f"{text} ({label})" for text, label in entities])
    enhanced_description = f"{description.strip()} {entity_summary}."

    return enhanced_description

# Remove duplicates
def remove_duplicates(artifacts):
    unique_titles = set()
    unique_artifacts = []

    for artifact in artifacts:
        title = artifact.get('title', '')
        if title not in unique_titles:
            unique_titles.add(title)
            unique_artifacts.append(artifact)
    return unique_artifacts

#update description for each artifacts
def update_artifact_descriptions(artifacts, nlp):
    for artifact in artifacts:
        original_description = artifact.get('description', '')
        #Enhance the original description using gen_desc func
        enhanced_description = generate_description(original_description, nlp)
        artifact['enhanced_description'] = enhanced_description
    return artifacts

#save new enhanced desc into a new json dile
def save_json(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

#input and output files
input_file = 'artifacts.json'
output_file = 'artifacts_FinalData.json'

# call functions
artifacts = load_json(input_file)
# remove_duplicates = remove_duplicates(artifacts)
updated_artifacts = update_artifact_descriptions(artifacts, nlp)
save_json(updated_artifacts, output_file)

**Phase 5**


In [ ]:
!pip install tensorflow keras numpy scikit-learn matplotlib


In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import cv2

In [ ]:
#https://medium.com/scrapehero/exploring-image-similarity-approaches-in-python-b8ca0a3ed5a3
def load_and_preprocess_image(img_path):
    img = cv2.imread(img_path)
    # Convert to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def compute_histogram(img):
    hist = cv2.calcHist([img], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

# Load and compute their histograms
def load_images_and_histograms(image_folder):
    image_paths = []
    histograms = []

    for img_file in os.listdir(image_folder):
        if img_file.endswith('.jpg'):
            img_path = os.path.join(image_folder, img_file)
            img = load_and_preprocess_image(img_path)
            hist = compute_histogram(img)
            image_paths.append(img_path)
            histograms.append(hist)

    return image_paths, histograms

#simple histogram comparison to find similarity
def find_top_similar_images(input_histogram, dataset_histograms, top_n=3):
    distances = []
    for hist in dataset_histograms:
        distance = cv2.compareHist(np.array(input_histogram, dtype=np.float32), np.array(hist, dtype=np.float32), cv2.HISTCMP_BHATTACHARYYA)  # Compute distance
        distances.append(distance)
    top_indices = np.argsort(distances)[:top_n]
    return top_indices

# Save images to a folder
def save_images(image_paths, save_folder):
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    for i, img_path in enumerate(image_paths):
        img = cv2.imread(img_path)
        save_path = os.path.join(save_folder, f"image_{i+1}.jpg")
        cv2.imwrite(save_path, img)
        print(f"Saved image: {save_path}")

image_folder = 'images'
save_folder = 'sim_images'
input_image_path = 'images/artifact_1.jpg'

# Load dataset images
image_paths, histograms = load_images_and_histograms(image_folder)

# Load and preprocess the input image
input_image = load_and_preprocess_image(input_image_path)
input_histogram = compute_histogram(input_image)

# Find the top 3 similar images
top_indices = find_top_similar_images(input_histogram, histograms)

# Save the selected images
top_similar_images = [image_paths[i] for i in top_indices]
save_images(top_similar_images, save_folder)


Saved image: sim_images/image_1.jpg
Saved image: sim_images/image_2.jpg
Saved image: sim_images/image_3.jpg


In [ ]:
# # https://www.kaggle.com/code/vincentman0403/find-similar-fruit-images-by-vgg
# #load and preprocess one image for vgg
# def load_and_preprocess_image(img_path, target_size=(224, 224)):
#     img = image.load_img(img_path, target_size=target_size)
#     img_data = image.img_to_array(img)
#     img_data = np.expand_dims(img_data, axis=0)
#     img_data = preprocess_input(img_data)
#     return img_data

# #load all images from folder
# def load_images(image_folder):
#     image_paths = []
#     images = []
#     # Loop through images in folder
#     for img in os.listdir(image_folder):
#         if img.endswith('.jpg'):
#             img_path = os.path.join(image_folder, img)
#             image_paths.append(img_path)
#             # Load and preprocess the image
#             img_data = load_and_preprocess_image(img_path)
#             images.append(img_data)
#     return images, image_paths

# #Extract features from images
# def extract_features(images):
#     model = VGG16(weights='imagenet', include_top=False, pooling='avg')
#     features = [model.predict(img)[0] for img in images]
#     return np.array(features)

# #cosine similarity between image features.
# def compute_cosine_similarity(features):
#     return cosine_similarity(features)

# # Find the top 3 similar images
# def find_top_similar_images(input_features, dataset_features, top_n=3):
#     similarities = cosine_similarity([input_features], dataset_features)[0]  # Calculate similarities directly
#     top_indices = np.argsort(-similarities)[1:top_n+1]  # Skip the first one which is the input itself
#     return top_indices


# #Display images directly using PIL.
# def display_images_with_pil(image_paths):
#     for img_path in image_paths:
#         img = Image.open(img_path)
#         img.show()

# #save images to folder
# def save_image(image_paths, save_folder):
#     # Create the folder if it doesn't exist
#     if not os.path.exists(save_folder):
#         os.makedirs(save_folder)
#     # Save each image
#     for i, img_path in enumerate(image_paths):
#         img = Image.open(img_path)
#         # Convert to 'RGB' to save as JPEG
#         if img.mode not in ('RGB', 'RGBA'):
#             img = img.convert('RGB')
#         save_path = os.path.join(save_folder, f"image_{i+1}.jpg")
#         img.save(save_path)
#         print(f"Saved image: {save_path}")


# image_folder = 'images'
# save_folder = 'sim_images'
# input_image_path = 'images/artifact_1.jpg'

# images, image_paths = load_images(image_folder)

# features = extract_features(images)

# input_image = load_and_preprocess_image(input_image_path)


# model = VGG16(weights='imagenet', include_top=False, pooling='avg')
# input_features = model.predict(input_image)[0]

# top_indices = find_top_similar_images(input_features, features)
# top_similar_images = [image_paths[i] for i in top_indices]

# save_image(top_similar_images, save_folder)



1/1 [==============================] - 1s 1s/step
Saved image: sim_images/image_1.jpg
Saved image: sim_images/image_2.jpg
Saved image: sim_images/image_3.jpg


**Phase 6**

In [ ]:
#simple chatbot for testing without ui
# Load artifact data from JSON file
# import json
def load_artifact_data(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        artifacts = json.load(f)
    return artifacts

#  Find and return information about artifacts
def find_artifact_info(artifacts, query):
    results = []
    query_lower = query.lower()
    found_titles = set()

    for artifact in artifacts:
        title = artifact.get('title', '').lower()
        description = artifact.get('description', '').lower()
        category = artifact.get('details', {}).get('Category', 'No Category')

        # Create a unified condition to avoid duplication
        if (query_lower in title or query_lower in description) and title not in found_titles:
            results.append({
                'id': artifact.get('id', 'No ID'),
                'title': artifact.get('title', 'No Title'),
                'description': artifact.get('description', 'No Description'),
                'category': category,
                'Present location': artifact.get('details', {}).get('Present location', 'No Location'),
                'Inventory number': artifact.get('details', {}).get('Inventory number', 'No Inventory Number'),
                'Dating': artifact.get('details', {}).get('Dating', 'No Dating'),
                'Archaeological Site': artifact.get('details', {}).get('Archaeological Site', 'No Site'),
                'Material': artifact.get('details', {}).get('Material', 'No Material'),
                'Technique': artifact.get('details', {}).get('Technique', 'No Technique'),
                'Height': artifact.get('details', {}).get('Height', 'No Height'),
                'Width': artifact.get('details', {}).get('Width', 'No Width'),
                'Depth': artifact.get('details', {}).get('Depth', 'No Depth')
            })
            found_titles.add(title)

    return results


#Interact with the chatbot testing
def chat_with_bot(artifacts):
  #Type 'exit' to end the chat
    while True:
      # Start the conversation
        query = input("You: ")

        if query.lower() == 'exit':
            break

        results = find_artifact_info(artifacts, query)

        if results:
            for info in results:
                print(f"\n---")
                print(f"Chatbot: As a historian, I can tell you about {info['title']} (ID: {info['id']}):")
                print("Description:")
                print(info['description'])
                print("Category:", info['category'])
                print("Present location:", info['Present location'])
                print("Inventory number:", info['Inventory number'])
                print("Dating:", info['Dating'])
                print("Archaeological Site:", info['Archaeological Site'])
                print("Material:", info['Material'])
                print("Technique:", info['Technique'])
                print("Height:", info['Height'])
                print("Width:", info['Width'])
                print("Depth:", info['Depth'])
        else:
            print("Chatbot: I couldn't understand your question.")
            print("Please include the exact title of an artifact or the name of a category in your query to know more about it.")

# Load artifact data
json_path = 'artifacts_FinalData.json'
artifacts = load_artifact_data(json_path)
chat_with_bot(artifacts)


You: mummy label

---
Chatbot: As a historian, I can tell you about Mummy label (ID: No ID):
Description:
From the 2nd century of our era, the Egyptians  produced documents of wood called "mummy labels". They contain short religious formulae mentioning the name, affiliation, occupation, age and the date of death of the deceased. This piece in the form of a sign was intended to be fixed to the mummy of Sarapammon, son of Harpalos. The Museum owns a second "label" belonging to the brother of Sarapammon (A.1974).
Category: MUMMY LABEL
Present location: KMKG - MRAH [07/003] BRUSSELS
Inventory number: A.1973
Dating: ROMAN PERIOD
Archaeological Site: TIHNA EL-GABAL
Material: WOOD
Technique: SCULPTURED; ENGRAVED
Height: 10.5 cm
Width: No Width
Depth: No Depth
You: mummy label

---
Chatbot: As a historian, I can tell you about Mummy label (ID: No ID):
Description:
From the 2nd century of our era, the Egyptians  produced documents of wood called "mummy labels". They contain short religious form